In [ ]:
import sys
sys.path.append('..')

In [ ]:
import pandas as pd
import random
from continuous_BCQ.utils import ReplayBuffer

In [ ]:
df = pd.read_csv("ibState0_BCQ.csv")

In [ ]:
df['done'] = 0

In [ ]:
df

In [ ]:
#"name": ["feed_c", "out_c", "feed_f", "out_f", "out_f_setting", "pressure", "reward"]

In [ ]:
df_1 = df

In [ ]:
df_1

## 标准化

In [ ]:
df_zscore = df_1.copy()
df_zscores = (df_zscore - df_zscore.mean()) / df_zscore.std()
df_zscores['done'] = 0
df_zscores['p'] = 0
df_zscores
# df_zscores = df_zscores.drop(df_zscores.index[-1])

## 数据集两两绑定（state和next_state）

In [ ]:
df_zscores_single = [x for x in range(0, len(df_zscores), 2)]
len(df_zscores_single)

## 划分验证集和训练集（差集）-------两两绑定

In [ ]:
df_zscores_single_eval = random.sample(df_zscores_single, 12500)

In [ ]:
len(df_zscores_single_eval)

In [ ]:
df_zscores_single_train = list(set(df_zscores_single) - set(df_zscores_single_eval))

In [ ]:
len(df_zscores_single_train)

In [ ]:
data = df_zscores.values

In [ ]:
print(data[0])
data[0][0:3]

In [ ]:
state_dim = 6
action_dim = 3

In [ ]:
replay_buffer_train = ReplayBuffer(state_dim, action_dim, device="cpu")
replay_buffer_eval = ReplayBuffer(state_dim, action_dim ,device="cpu")

In [ ]:
for t in range(0, len(data), 2):
    print(t)
    if t in df_zscores_single_train:
        state = data[t][3:-3]
        action = data[t][0:3]
        next_state = data[t+1][3:-3]
        reward = data[t+1][-2]
        done = data[t+1][-1]
        replay_buffer_train.add(state, action, next_state, reward, done)
        print("replay_buffer_train_add", t)
    elif t in df_zscores_single_eval:
        state = data[t][3:-3]
        action = data[t][0:3]
        next_state = data[t+1][3:-3]
        reward = data[t+1][-2]
        done = data[t+1][-1]
        replay_buffer_eval.add(state, action, next_state, reward, done)
        print("replay_buffer_eval_add", t)
    else:
        pass

In [ ]:
# # 训练集
# for t in range(len(data_train[:-1])):
#     state = data[t][0:-4]
#     action = data[t][-2]
#     next_state = data[t+1][0:-4]
#     reward = data[t+1][-3]
#     done = data[t][-1]
#     replay_buffer_train.add(state, action, next_state, reward, done)
# # 验证集
# for t in range(len(data_eval[:-1])):
#     state = data[t][0:-4]
#     action = data[t][-2]
#     next_state = data[t+1][0:-4]
#     reward = data[t+1][-3]
#     done = data[t][-1]
#     replay_buffer_eval.add(state, action, next_state, reward, done)

In [ ]:
buffer_name = "replay_buffer_ib"
replay_buffer_train.save(f"./buffers/{buffer_name}_train")
replay_buffer_eval.save(f"./buffers/{buffer_name}_eval")